In [ ]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

datos = pd.read_csv('Seqs_and_Computed_Descriptors/path...')

X=datos.drop(['PRED'], axis=1)
y = datos['PRED']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42)


# Classification assessment using 80% and 20% of data for training and testing, respectively.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix

model = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2,
                                min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0,
                                bootstrap=True, oob_score=False, n_jobs=None, random_state=0, verbose=0, warm_start=False, class_weight=None,
                                ccp_alpha=0.0, max_samples=None).fit(X_train, y_train)


def confusion_matrix_scorer(model, X_train, y_train):
        y_pred = model.predict(X_train)
        cm = confusion_matrix(y_train, y_pred)
        return {'tn': cm[0, 0], 'fp': cm[0, 1],
                'fn': cm[1, 0], 'tp': cm[1, 1]}
        
cv_results = cross_validate(model, X_train, y_train, cv=10,
                            scoring=confusion_matrix_scorer)
# Getting the test set true positive scores
TP = cv_results['test_tp'].mean()

# Getting the test set false negative scores
FN = cv_results['test_fn'].mean()

# Getting the test set false positive scores
FP = cv_results['test_fp'].mean()

# Getting the test set true negative scores
TN = cv_results['test_tn'].mean()

# Model generation from the training with 100% of data

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix

model = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2,
                                min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0,
                                bootstrap=True, oob_score=False, n_jobs=None, random_state=42, verbose=0, warm_start=False, class_weight=None,
                                ccp_alpha=0.0, max_samples=None).fit(X, y) #Fitted model


                         
# Save the model using the 100% of data
import joblib
joblib.dump(model, 'Model/model.pkl')

#Cross-validation
def confusion_matrix_scorer(model, X, y):
        y_pred = model.predict(X)
        cm = confusion_matrix(y, y_pred)
        return {'tn': cm[0, 0], 'fp': cm[0, 1],
                'fn': cm[1, 0], 'tp': cm[1, 1]}
        
cv_results = cross_validate(model, X, y, cv=10,
                            scoring=confusion_matrix_scorer)
# Getting the test set true positive scores
TP = cv_results['test_tp'].mean()

# Getting the test set false negative scores
FN = cv_results['test_fn'].mean()

# Getting the test set false positive scores
FP = cv_results['test_fp'].mean()

# Getting the test set true negative scores
TN = cv_results['test_tn'].mean()

# Perfomance measurement computations

In [ ]:
####TRAINING###

#pred_train=model.predict(X_train)
pred_train_all =model.predict(X)

acurracy = (TP+TN) / (TP+TN+FP+FN)
F1_score = 2*TP / ((2*TP) + (FP + FN))
precision = TP / (TP + FP)
specificity = TN / (FP + TN)
sensitivity_recall = TP / (TP + FN)

import math 
MCC = ((TP*TN) - (FP*FN)) / math.sqrt(((TP+FP)*(TP+FN))*((TN+FP)*(TN+FN)))

print("Accuracy: ", acurracy)
print("F1_score: ", F1_score)
print("Precision: ", precision)
print("Specificity: ", specificity)
print("Sensitivity/Recall: ", sensitivity_recall)
print("MCC: ", MCC)

from sklearn.metrics import roc_auc_score
roc = roc_auc_score(y_train, model.predict_proba(X_train), multi_class='ovr')
print("ROC_AUC: ", roc)

#Plot ROC curve training
tpr = TP / (TP + FN)
fpr = FP / (FP + TN)

import matplotlib.pyplot as plt
def plot_roc_curve(fpr, tpr):
    #plt.plot(fpr, tpr, color='#00FF74', label='ROC')
    plt.plot(fpr, tpr, color='#9300FF', label='ROC')
    #plt.plot(fpr, tpr, color='#FFC300', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Training = 100%)')
    #plt.title('ROC Curve (Training = 100%)')
    plt.legend()
    #plt.savefig('ROC_all.png', dpi=1200)
    plt.show()
    
plot_roc_curve(fpr, tpr)

In [ ]:
####TESTING###
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

pred_test=model.predict(X_test)

conf = confusion_matrix(y_test, pred_test)
TPt = conf[1, 1]
FPt = conf[0, 1]
TNt = conf[0, 0]
FNt = conf[1, 0]

acurracy = (TPt+TNt) / (TPt+TNt+FPt+FNt)
F1_score = 2*TPt / ((2*TPt) + (FPt + FNt))
precision = TPt / (TPt + FPt)
specificity = TNt / (FPt + TNt)
sensitivity_recall = TPt / (TPt + FNt)

import math 
MCC = ((TPt*TNt) - (FPt*FNt)) / math.sqrt(((TPt+FPt)*(TPt+FNt))*((TNt+FPt)*(TNt+FNt)))

print("Accuracy: ", acurracy)
print("F1_score: ", F1_score)
print("Precision: ", precision)
print("Specificity: ", specificity)
print("Sensitivity/Recall: ", sensitivity_recall)
print("MCC: ", MCC)

from sklearn.metrics import roc_auc_score
roc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr')
print("ROC_AUC: ", roc)

#Plot ROC curve testing
tpr_t = TPt / (TPt + FNt)
fpr_t = FPt / (FPt + TNt)

def plot_roc_curve(fpr_t, tpr_t):
    plt.plot(fpr_t, tpr_t, color='#FF0000', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Testing = 20%)')
    plt.legend()
    #plt.savefig('ROC_20.png', dpi=1200)
    plt.show()
    
plot_roc_curve(fpr_t, tpr_t)

In [ ]:
#Confusion matrix
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
#ConfusionMatrixDisplay.from_predictions(y_test, pred_test)
#ConfusionMatrixDisplay.from_predictions(y_train, pred_train)
ConfusionMatrixDisplay.from_predictions(y, pred_train_all)
#plt.title('Confusion Matrix (Training [CV-10] = 80%)')
#plt.title('Confusion Matrix (Testing = 20%)')
plt.title('Confusion Matrix (Training [CV-10] = 100%)')
plt.savefig('matrix_all.png', dpi=1200)
plt.show()

# Proteome predictions

In [ ]:
import pandas as pd
X1 = pd.read_csv('Seqs_and_Computed_Descriptors/path...')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X1)
X1 = scaler.transform(X1)

In [ ]:
import numpy as np
import joblib


model = joblib.load('Model/model.pkl')
prediction = model.predict(X1)
pred = model.predict_proba(X1)

In [ ]:
#Save predictions
save_pred = pd.DataFrame(pred).to_csv("Predictions/file_name.csv", index=False)
save_pred